In [ ]:
import functools
import geopy
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas.tools.rplot as rplot
import pandas as pd
import pyproj
import requests
from scipy import signal
%pylab inline

In [ ]:
rides = pd.read_csv('../csv_dump/rides.csv')
readings = pd.read_csv('../csv_dump/readings.csv')

In [ ]:
#EPSG:3435
#NAD83 / Illinois East (ftUS) (Google it)
#Note that this this projection takes arguments in the form (lon, lat),
#presumably to agree with the (x, y) output
NAD83 = pyproj.Proj('+init=EPSG:3435')

In [ ]:
location_query = 'http://router.project-osrm.org/nearest?loc='

In [ ]:
print readings.start_lat[0]
print readings.start_lon[0]

In [ ]:
print rides.columns
print readings.columns

In [ ]:
# readings.sort(columns = ['ride_id', 'id'], inplace=True)
readings.sort_values(by=['ride_id', 'id'], inplace=True)

In [ ]:
nearest_street = requests.get(location_query + str(readings.start_lat[0]) + ',' + str(readings.start_lon[0]))

In [ ]:
nearest_street.json()['name']

In [ ]:
# def string_to_array(str_array):
#     if str_array.strip() == '---':
#         return pd.Series()
#     if str_array[:2] == '- ':
#         return pd.Series(float(str_array[2:]))
#     return pd.Series([float(i) for i in str_array[5:-1].split('\n- ')])




        
# def string_to_array(str_array):
#     if str_array != []:
#         return_array = []
#         for i, char in enumerate(str_array[3:].strip().split('\n-')[:-1]):
#             try:
#                 float(char)
#             except:
#                 np.NaN

#     return pd.Series([float(i) for i in str_array.strip()[3:].split('\n-')])

# for axis in ['x', 'y', 'z']:
#     split_accel = readings['acceleration_' + axis].apply(string_to_array).stack()
# (apply(Series, 1)
# for i in readings.acceleration_x:
#     assert(i.strip().split('\n-')[0] == '---'s = df['ID'].apply(Series,1).stack())
# a_thing = [float(i) for i in readings.acceleration_x[0].split('\n-')[1:]]

In [ ]:
# for axis in ['x', 'y', 'z']:
#     split_accel = readings['acceleration_' + axis].apply(string_to_array)
#     split_accel.columns = ['acceleration_' + axis + '_' + str(l) for l in split_accel.columns]
#     readings = readings.join(split_accel)

In [ ]:
def string_to_array(str_array):
    if str_array.strip() == '---':
        return pd.Series()
    if str_array[:2] == '- ':
        return pd.Series(float(str_array[2:]))
    return np.array([float(i) for i in str_array[5:-1].split('\n- ')])

for axis in ['x', 'y', 'z']:
    split_accel = readings['acceleration_' + axis].apply(string_to_array)
    readings['num_accel_' + axis] = split_accel

In [ ]:
readings.loc[0, ['start_lon', 'start_lat']]

In [ ]:
NAD83 = pyproj.Proj('+init=EPSG:3435')
def proj_apply(row, proj):
    x,y = proj(*row)
    return pd.Series({'x':x, 'y':y})
NAD83_apply = functools.partial(proj_apply, proj=NAD83)
# NAD83_apply = lambda x: pd.SeriesNAD83(*x)

In [ ]:
# lambdafunc = lambda x: pd.Series([x['mytime'].hour,
#                                   x['mydate'].isocalendar()[1],
#                                   x['mydate'].weekday()])

# newcols = df.apply(lambdafunc, axis=1)
# newcols.columns = ['hour', 'weekday', 'weeknum']
# newdf = df.join(newcols)

In [ ]:
start_xy = readings.loc[:, ['start_lon', 'start_lat']].apply(NAD83_apply, axis=1)
end_xy = readings.loc[:, ['end_lon', 'end_lat']].apply(NAD83_apply, axis=1)

In [ ]:
start_xy.columns = ('start_x', 'start_y')
end_xy.columns = ('end_x', 'end_y')

In [ ]:
readings = readings.join(start_xy)
readings = readings.join(end_xy)

In [ ]:
plot = rplot.RPlot(readings, x='start_x', y='start_y')
plot.add(rplot.GeomPoint(size=0.1, colour=rplot.ScaleGradient))

In [ ]:
help(rplot.ScaleGradient('size'))

In [ ]:
plot.add(rplot.GeomPoint(size=80.0, colour=rplot.ScaleRandomColour('day'), shape=rplot.ScaleShape('size'), alpha=1.0))


In [ ]:

plot.add(rplot.GeomPoint(size=80.0, colour=rplot.ScaleRandomColour('day'), shape=rplot.ScaleShape('size'), alpha=1.0))

In [ ]:
readings.plot('start_x', 'start_y', kind='scatter')
plt.axis('equal')
config = plt.gcf()
config.set_size_inches((15, 15))
plt.show()



In [ ]:
"""It looks almost exactly the same if you use raw lon/lat. """

# readings.plot('start_lon', 'start_lat', kind='scatter')
# plt.axis('equal')
# config = plt.gcf()
# config.set_size_inches((15, 25))
# plt.show()

In [ ]:
subset_readings = readings[['ride_id', 'num_accel_x', 'num_accel_y', 'num_accel_z']]

In [ ]:
sum(subset_readings.ride_id == 1)

In [ ]:
ride_1_accels = pd.DataFrame()
for axis in ['x', 'y', 'z']:
    ride_1_accels[axis] = np.concatenate(list(readings['num_accel_' + axis][:587]), axis=1)

In [ ]:
readings['total_readings'] = readings.num_accel_x.apply(len)
readings['duration'] = readings.end_time - readings.start_time

In [ ]:
readings.plot(x='duration', y='total_readings', kind='scatter')
plt.show()

In [ ]:
(readings.total_readings / readings.duration).plot(kind='hist')

In [ ]:
readings.total_readings.plot(kind='hist', bins=1000)
plt.show()

In [ ]:
current_index = 0
for i in range(20):
    print current_index
    current_index += len(readings.num_accel_z[i])
    f, Pxx_den = signal.periodogram(readings.num_accel_x[i])
    plt.semilogy(f, Pxx_den)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
config.set_size_inches = (15, 10)
plt.show()

In [ ]:
ride_1_accels.plot(alpha=0.6)
# ride_1_accels.iloc[400:1000, :].plot(alpha=0.6)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
config.set_size_inches = (15, 10)
plt.show()

In [ ]:
readings.ix[readings.ride_id == 1.0,['num_accel_x']].plot()

In [ ]:
subset_readings.index = readings['id']

In [ ]:
small_grouped_readings = subset_readings.iloc[:5, :].groupby('ride_id')

In [ ]:
test_df = subset_readings.num_accel_x.iloc[:5]

In [ ]:
test_df

In [ ]:
def append_col(col):
    return np.concatenate(list(col), axis=1)
def cols_to_long_array(df):
#     append_col = lambda col: np.concatenate(col, axis=1)
    return df.apply(append_col, axis=0, reduce=True)

small_grouped_readings.aggregate(np.concatenate, axis=1)

In [ ]:
pd.DataFrame.apply

In [ ]:
grouped_readings = subset_readings.groupby(['ride_id'])

In [ ]:
np.append(np.array([0, 1, 5]), np.array([6, 9, 12]))

In [ ]:
# np.append(subset_readings.num_accel_x.iloc[0], subset_readings.num_accel_x.iloc[1])

In [ ]:
small_grouped_readings.aggregate(np.append)
# small_grouped_readings.agg({'num_accel_x': np.append})

In [ ]:
test_array = readings.num_accel_x[0]

In [ ]:
readings.loc[readings.ride_id == 1, :].plot(x='acceleration_x', y='acceleration_y', kind='scatter')
plt.show()

In [ ]:
readings['total_readings'] = readings.acceleration_x.apply(lambda x: x.count('\n'))
# total_readings = pd.DataFrame({'total_readings':
#                                [readings.acceleration_x[i].count('\n') for i in range(readings.shape[0])]})

In [ ]:
plt.hist(total_readings['total_readings'], bins=100)
plt.show()